In [1]:
import os

from speechbrain.pretrained import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb",run_opts={"device":"cuda"})
from IPython.display import Audio, display

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [2]:
from tqdm import tqdm

def verify_similarity(orig, hat):
   score, prediction = verification.verify_files(orig,hat) # Different Speakers
   return score, prediction 

def rename(aud_path, item_name):
   # Perform temporary renaming by adding temp so that no audio files have the same name
   os.rename(f"{aud_path}/{item_name}_source1hat.wav",f"{aud_path}/{item_name}_source2hat_temp.wav")
   os.rename(f"{aud_path}/{item_name}_source2hat.wav",f"{aud_path}/{item_name}_source1hat.wav")
   os.rename(f"{aud_path}/{item_name}_source2hat_temp.wav",f"{aud_path}/{item_name}_source2hat.wav")
   
   os.rename(f"{aud_path}/{item_name}_source1hat_up.wav",f"{aud_path}/{item_name}_source2hat_up_temp.wav")
   os.rename(f"{aud_path}/{item_name}_source2hat_up.wav",f"{aud_path}/{item_name}_source1hat_up.wav")
   os.rename(f"{aud_path}/{item_name}_source2hat_up_temp.wav",f"{aud_path}/{item_name}_source2hat_up.wav")


def run_renamer(aud_path):
   item_set = set()
   files = os.listdir(aud_path)
   for filename in tqdm(files, total=len(files)):
      item_name = filename.split('_')[0]
      # item_no = int(item_name.split('item')[1])
      
      if item_name not in item_set:
         scores = 0
         for i in range(1,3):
            orig = f"{aud_path}/{item_name}_source{i}.wav"
            hat = f"{aud_path}/{item_name}_source{i}hat.wav"
            score, pred = verify_similarity(orig, hat)
            # display(Audio(orig))
            # display(Audio(hat))
            scores+=score 
         is_match = False
         if scores>=1.4:
            is_match=True
            # print(f"{item_name} match:{is_match}")
         if not is_match:
            rename(aud_path, item_name)
         item_set.add(item_name)
      else:
         continue


configs = ["standard_model"]
for config in configs:
   aud_path = f"../results/{config}/audio_results"
   run_renamer(aud_path)

100%|█████████▉| 21000/21001 [08:38<00:00, 40.49it/s]


RuntimeError: Error opening 'New Text Document.txt_source1.wav': System error.